# 🧩 Роли и компоненты MAS в LLM-системах

Мультиагентные системы (MAS) с использованием больших языковых моделей (LLM) организуются по ролям и структурам, аналогично реальным командам. Ниже описаны 4 ключевых элемента: планировщик, рабочие агенты, память (memory) и протокол общения.

---

## 1. Планировщик (Planner)

Планировщик — это агент, который:
- анализирует цель задачи;
- разбивает её на подзадачи;
- назначает исполнителей (агентов);
- контролирует последовательность выполнения.

📌 Используется в системах:
- AutoGen (`PlannerAgent`)
- CrewAI (`Crew` и `TaskPlanner`)
- LangGraph (`State transitions` как implicit планирование)

---

## 2. Рабочие агенты (Worker Agents)

Рабочие агенты — это LLM-экземпляры с конкретной ролью. Каждый специализируется на:
- написании текста;
- анализе данных;
- генерации кода;
- проверке выводов.

Каждый агент имеет:
- инструкции (роль);
- доступ к инструментам (иногда через tool calling);
- способность обмениваться сообщениями.

---

## 3. Память (Memory)

Memory — это состояние или хранилище, которое:
- сохраняет историю взаимодействий;
- позволяет агентам "помнить", что уже было сделано;
- помогает избежать повторений или ошибок.

📌 Реализации:
- ConversationSummaryMemory (LangChain)
- SharedWorkspace (AutoGen)
- ChatHistory (CrewAI)


---

## 4. Протокол (Protocol / Coordination Mechanism)

Протокол — это механизм координации, который:
- управляет взаимодействием между агентами;
- определяет порядок общения;
- решает, кто когда говорит и кто принимает финальные решения.

📌 Примеры:
- GroupChat и GroupChatManager в AutoGen
- LangGraph использует переходы по состояниям
- CrewAI — pipeline-передача задачи от агента к агенту


---

## 🧠 Визуальное сравнение ролей

| Компонент      | Назначение                         | Пример в фреймворке |
|----------------|------------------------------------|---------------------|
| Планировщик    | Разбивка задачи, координация       | AutoGen, CrewAI     |
| Рабочий агент  | Выполнение подзадач                | LangChain Agents    |
| Память         | Хранение истории, контекста        | LangChain Memory    |
| Протокол       | Определение порядка взаимодействий | AutoGen GroupChat   |

---



# 💡 Как агенты общаются между собой?


### **Кооперация и конкуренция агентов в мультиагентных системах: архитектуры и стратегии**
---

<img src="/Users/dapavlik10/Downloads/llm_agents_chatbot/pics/types.png" width="25%">


[Multi-Agent Collaboration Mechanisms: A Survey of LLMs](https://arxiv.org/pdf/2501.06322)


### В мультиагентных системах (МАС) возможны разные типы взаимодействия между автономными агентами – от полного сотрудничества до жесткого соперничества. 
---
## 1. Кооперация (Cooperation) 
---
Кооперация подразумевает, что цели агентов согласованы, и они могут скоординированно работать ради общей выгоды. Исследования по кооперации агентов  сфокусированы на том, как добиться эффективной координации, когда интересы агентов совпадают, и даже как наладить сотрудничество, если совместная работа потенциально выгодна, но присутствует риск конфликта интересов. 

---

Агенты работают вместе, чтобы достичь общей цели.

🔍 Особенности:
- 📤 Обмен информацией и результатами.
- 📦 Распределение задач между агентами.
- 🔗 Согласование действий.
- 🎯 Общая цель у всех участников.

[A REVIEW OF COOPERATION IN MULTI-AGENT LEARNING](https://arxiv.org/pdf/2312.05162)

**Сделаем агентов-врачей и на их примере посмотрим, как это работает** 

In [2]:
# мы работаем с mistral nemo 12b локально, обращаясь к http://localhost:11434/v1

In [20]:
llm_config = {
    'model': 'mistral-nemo:12b-instruct-2407-q3_K_L',
    'base_url': 'http://localhost:11434/v1',
    'api_key': 'ollama',
    'cache_seed': None,
    'temperature': 0.2
}

In [39]:
from autogen import AssistantAgent, GroupChat, GroupChatManager, UserProxyAgent

# Агент 1: Исследователь
researcher = AssistantAgent(
    name="Researcher",
    llm_config=llm_config,
    system_message=(
        "Ты медицинский исследователь. Ищи и кратко описывай медицинскую информацию по запросу: "
        "симптомы, статистику, причины болезни. Не составляй диагнозов или лечения."
    )
)

# Агент 2: Врач
doctor = AssistantAgent(
    name="Doctor",
    llm_config=llm_config,
    system_message=(
        "Ты врач. Используй данные от исследователя и составь детальный план лечения и рекомендации."
    )
)

# Агент 3: Редактор
editor = AssistantAgent(
    name="Editor",
    llm_config=llm_config,
    system_message=(
        "Ты медицинский редактор. Оформи текст от других агентов в виде официального медицинского отчёта."
    )
)

# Агент-инициатор (User)
user = UserProxyAgent(
    name="User",
    code_execution_config=False,
    human_input_mode="NEVER"
)

# Групповой чат между агентами
group_chat = GroupChat(
    agents=[user, researcher, doctor, editor],
    messages=[],
    max_round=10
)

# Менеджер, управляющий диалогом
manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config
)

# Запуск задачи
user.initiate_chat(
    manager,
    message=(
        "Подготовьте совместный медицинский отчёт для пациента с диагнозом: "
        "бронхиальная астма. Сначала исследователь предоставляет медицинские данные, "
        "врач составляет план лечения, редактор оформляет результат."
    )
)

User (to chat_manager):

Подготовьте совместный медицинский отчёт для пациента с диагнозом: бронхиальная астма. Сначала исследователь предоставляет медицинские данные, врач составляет план лечения, редактор оформляет результат.

--------------------------------------------------------------------------------
[autogen.oai.client: 06-23 11:25:19] {696} WARNING - Model mistral-nemo:12b-instruct-2407-q3_K_L is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.

Next speaker: Researcher

[autogen.oai.client: 06-23 11:25:59] {696} WARNING - Model mistral-nemo:12b-instruct-2407-q3_K_L is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
Researcher (to chat_manager):

**Медицинский отчет**

*Пациент:* [Имя Фамилия]

*Диагноз:* Бронхиальная астма

**Медицинская информация:**

*Симптомы:*
- Одышка


ChatResult(chat_id=None, chat_history=[{'content': 'Подготовьте совместный медицинский отчёт для пациента с диагнозом: бронхиальная астма. Сначала исследователь предоставляет медицинские данные, врач составляет план лечения, редактор оформляет результат.', 'role': 'assistant', 'name': 'User'}, {'content': '**Медицинский отчет**\n\n*Пациент:* [Имя Фамилия]\n\n*Диагноз:* Бронхиальная астма\n\n**Медицинская информация:**\n\n*Симптомы:*\n- Одышка\n- Кашель, особенно ночью или утром\n- Затрудненное дыхание\n- Свистящие звуки при дыхании\n- Чувство сдавливания в груди\n\n*Статистика:*\n- Бронхиальная астма является хроническим заболеванием дыхательных путей, которое поражает более 300 миллионов человек во всем мире.\n- В России от бронхиальной астмы страдают около 7 миллионов человек.\n\n*Причины:*\n- Наследственность\n- Аллергические реакции на пыльцу, плесень, домашнюю пыль и т.д.\n- Инфекционные заболевания дыхательных путей в раннем детстве\n- Курение и воздействие загрязнителей воздуха\

##  2. Конкуренция (Competition)


Агенты действуют независимо и стремятся превзойти друг друга.

🔍 Особенности:
- 💥 Борьба за ресурсы или внимание.
- 🧮 Оценка по метрикам (скорость, точность).
- 🧬 Эволюционный отбор лучших решений.
- 🎯 Разные цели или стратегии.

💡 Примеры:
- Несколько агентов генерируют разные версии ответа — побеждает лучший.
- Переводчики-соперники: у кого стиль лучше?

---


In [38]:
from autogen import AssistantAgent, UserProxyAgent

# Врачи
doctor_a = AssistantAgent(
    name="DoctorA",
    llm_config=llm_config,
    system_message=(
        "Ты опытный врач, применяющий проверенные методы лечения. "
        "Фокусируйся на безопасности и стандартах."
    )
)

doctor_b = AssistantAgent(
    name="DoctorB",
    llm_config=llm_config,
    system_message=(
        "Ты прогрессивный врач, применяющий инновационные и современные подходы. "
        "Не бойся предлагать нестандартные решения, если они обоснованы."
    )
)

# Судья
judge = AssistantAgent(
    name="Judge",
    llm_config=llm_config,
    system_message=(
        "Ты медицинский эксперт. Проанализируй два плана лечения и выбери лучший, объяснив почему."
    )
)

# Пользователь
user = UserProxyAgent(
    name="User",
    code_execution_config=False,
    human_input_mode="NEVER",
)

# Медицинский случай
case_description = (
    "Пациент: мужчина, 58 лет. Диагноз: сахарный диабет 2 типа, гипертония, лишний вес. "
    "Разработайте индивидуальный план лечения и изменения образа жизни."
)

# Получаем ответ от DoctorA
response_a = doctor_a.generate_reply(
    messages=[{"role": "user", "content": case_description}]
)

# Получаем ответ от DoctorB
response_b = doctor_b.generate_reply(
    messages=[{"role": "user", "content": case_description}]
)

# Создаем сравнение для Judge
comparison_prompt = (
    f"Вот два плана лечения для одного пациента:\n\n"
    f"--- План от DoctorA ---\n{response_a}\n\n"
    f"--- План от DoctorB ---\n{response_b}\n\n"
    f"Проанализируй оба плана и выбери лучший. Объясни свой выбор."
)

# Получаем решение от Judge
final_decision = judge.generate_reply(
    messages=[{"role": "user", "content": comparison_prompt}]
)

# Вывод
print("\n=== 🧑‍⚕️ План от DoctorA ===\n")
print(response_a)

print("\n=== 🧑‍⚕️ План от DoctorB ===\n")
print(response_b)

print("\n=== ⚖️ Решение судьи ===\n")
print(final_decision)

[autogen.oai.client: 06-23 00:05:30] {696} WARNING - Model mistral-nemo:12b-instruct-2407-q3_K_L is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
[autogen.oai.client: 06-23 00:06:33] {696} WARNING - Model mistral-nemo:12b-instruct-2407-q3_K_L is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
[autogen.oai.client: 06-23 00:07:53] {696} WARNING - Model mistral-nemo:12b-instruct-2407-q3_K_L is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.

=== 🧑‍⚕️ План от DoctorA ===

Для разработки индивидуального плана лечения и изменения образа жизни для данного пациента, я предлагаю следующее:

1. Контроль сахарного диабета 2 типа:
* Рекомендуется придерживаться диеты с низким гликемическим ин

## 🔁 3. Коопетиция (повторная кооперация)

Комбинация: сначала кооперация → затем конкуренция → затем снова кооперация.

🔍 Особенности:
- ♻️ Итеративный цикл.
- 🧠 Интеграция лучших решений после соревнования.
- 📈 Обучение на ошибках.
- 🛠️ Адаптация поведения агентов.

[A review of cooperation in multi-agent learning](https://arxiv.org/abs/2312.05162)

Повторная кооперация представляет особый интерес, поскольку при многократных встречах между одними и теми же агентами у них появляется
возможность выработать доверие, репутацию и устойчивые модели поведения. 

В однократной игре (например,  дилемма заключённого) рациональные эгоистичные агенты обычно выбирают предательство, но если эту игру повторять многократно, ситуация меняется. 

Повторяющиеся взаимодействия позволяют реализовать механизмы вознаграждения и наказания, стимулирующие долгосрочную кооперацию (например, через вознаграждение сотрудничающих агентов или «наказание» за жадное поведение), которые отсутствуют в одноразовых контактах. 

Современные исследования с обучающимися агентами подтверждают:
-   наличие достаточно длинной памяти и учёт будущих выгод существенно повышают вероятность устойчивого сотрудничества.


In [40]:
from autogen import AssistantAgent, GroupChat, GroupChatManager, UserProxyAgent

# Агент 1: Консервативный врач
doctor_a = AssistantAgent(
    name="DoctorA",
    llm_config=llm_config,
    system_message=(
        "Ты опытный врач, следуешь традиционным подходам. Предложи план лечения пациента."
    )
)

# Агент 2: Инновационный врач
doctor_b = AssistantAgent(
    name="DoctorB",
    llm_config=llm_config,
    system_message=(
        "Ты прогрессивный врач. Придумай инновационный, но безопасный план лечения пациента."
    )
)

# Агент 3: Синтезатор
mediator = AssistantAgent(
    name="Mediator",
    llm_config=llm_config,
    system_message=(
        "Ты медицинский аналитик. Твоя задача — изучить два предложенных плана лечения и составить "
        "оптимальный объединённый план, используя лучшие стороны каждого."
    )
)

# Пользователь
user = UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    code_execution_config=False
)

# Групповой чат
group_chat = GroupChat(
    agents=[user, doctor_a, doctor_b, mediator],
    messages=[],
    max_round=12
)

manager = GroupChatManager(groupchat=group_chat, llm_config=llm_config)

# Запуск
user.initiate_chat(
    manager,
    message=(
        "Пациент: 60 лет, диагноз — ишемическая болезнь сердца и гипертония. "
        "DoctorA и DoctorB, предложите каждый свой план лечения. "
        "Затем Mediator создаст единый объединённый план, объединив лучшие идеи."
    )
)

User (to chat_manager):

Пациент: 60 лет, диагноз — ишемическая болезнь сердца и гипертония. DoctorA и DoctorB, предложите каждый свой план лечения. Затем Mediator создаст единый объединённый план, объединив лучшие идеи.

--------------------------------------------------------------------------------
[autogen.oai.client: 06-23 11:44:24] {696} WARNING - Model mistral-nemo:12b-instruct-2407-q3_K_L is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.

Next speaker: DoctorA

[autogen.oai.client: 06-23 11:46:11] {696} WARNING - Model mistral-nemo:12b-instruct-2407-q3_K_L is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
DoctorA (to chat_manager):

DoctorA:

План лечения пациента с ишемической болезнью сердца (ИБС) и гипертонией включает в себя комплексный подход к управлению рисками, моди

ChatResult(chat_id=None, chat_history=[{'content': 'Пациент: 60 лет, диагноз — ишемическая болезнь сердца и гипертония. DoctorA и DoctorB, предложите каждый свой план лечения. Затем Mediator создаст единый объединённый план, объединив лучшие идеи.', 'role': 'assistant', 'name': 'User'}, {'content': 'DoctorA:\n\nПлан лечения пациента с ишемической болезнью сердца (ИБС) и гипертонией включает в себя комплексный подход к управлению рисками, модификации образа жизни и фармакологической терапии.\n\n1. Модификация образа жизни:\n\t* Рекомендуется соблюдать диету с низким содержанием жиров и холестерина (например, средиземноморская диета).\n\t* Пациенту следует бросить курить, если он курит.\n\t* Рекомендуется регулярные физические упражнения не менее 30 минут в день, 5 дней в неделю.\n\t* Пациент должен поддерживать здоровый вес и уровень сахара в крови.\n2. Фармакологическая терапия:\n\t* Блокаторы кальциевых каналов (например, амлодипин) для контроля артериального давления и профилактики п

# Архитектуры мультиагентных систем: **централизованные**, **распределённые** и **иерархические**

### Три классических структуры мультиагентных систем:
- 🔹 Централизованная
- 🔹 Распределённая
- 🔹 Иерархическая

Контекст задачи (общий для всех трёх):

Система агентов должна подготовить технический отчёт по теме “**Будущее искусственного интеллекта в медицине**”.

Задача включает:
- 🔍 Исследование темы
- 🏗 Структурирование
- ✍ Написание текста
- 🧾 Редактирование отчёта

----

### 1️⃣ **Централизованная структура (Centralized)**

👤 Один центральный агент (менеджер) получает команду и назначает задачи подчинённым агентам.

In [41]:
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager

# Агент-менеджер (центр)
manager = AssistantAgent(
    name="Manager",
    llm_config=llm_config,
    system_message="Ты координатор. Делегируй задачи другим агентам и собери финальный отчёт."
)

# Исполнители
researcher = AssistantAgent(name="Researcher", llm_config=llm_config,
    system_message="Ты исследователь. Собери факты о применении ИИ в медицине.")
writer = AssistantAgent(name="Writer", llm_config=llm_config,
    system_message="Ты автор. Напиши текст по результатам исследования.")
editor = AssistantAgent(name="Editor", llm_config=llm_config,
    system_message="Ты редактор. Преобразуй текст в законченный отчёт.")

# Пользователь
user = UserProxyAgent(name="User", human_input_mode="NEVER", code_execution_config=False)

group_chat = GroupChat(
    agents=[user, manager, researcher, writer, editor],
    messages=[],
    max_round=12
)

chat_manager = GroupChatManager(groupchat=group_chat, llm_config=llm_config)

user.initiate_chat(chat_manager, message=(
    "Подготовьте совместный отчёт по теме 'Будущее ИИ в медицине'. Менеджер координирует действия других агентов."
))

User (to chat_manager):

Подготовьте совместный отчёт по теме 'Будущее ИИ в медицине'. Менеджер координирует действия других агентов.

--------------------------------------------------------------------------------
[autogen.oai.client: 06-23 12:06:17] {696} WARNING - Model mistral-nemo:12b-instruct-2407-q3_K_L is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.

Next speaker: Manager

[autogen.oai.client: 06-23 12:06:51] {696} WARNING - Model mistral-nemo:12b-instruct-2407-q3_K_L is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
Manager (to chat_manager):

В качестве координатора, я делегирую следующие задачи для подготовки отчета о будущем ИИ в медицине:

1. Агент 1: Исследуйте и напишите раздел "Введение" и "Текущее состояние ИИ в медицине". Опишите текущие применения ИИ в области

ChatResult(chat_id=None, chat_history=[{'content': "Подготовьте совместный отчёт по теме 'Будущее ИИ в медицине'. Менеджер координирует действия других агентов.", 'role': 'assistant', 'name': 'User'}, {'content': 'В качестве координатора, я делегирую следующие задачи для подготовки отчета о будущем ИИ в медицине:\n\n1. Агент 1: Исследуйте и напишите раздел "Введение" и "Текущее состояние ИИ в медицине". Опишите текущие применения ИИ в области здравоохранения, такие как диагностика заболеваний, лечение и профилактика.\n2. Агент 2: Исследуйте и напишите раздел "Будущее возможностей ИИ в медицине". Обсудите потенциальные применения ИИ в здравоохранении, такие как персонализированная медицина, прогнозирование эпидемий и разработка новых лекарств.\n3. Агент 3: Исследуйте и напишите раздел "Этические аспекты и вызовы". Рассмотрите этические вопросы, связанные с использованием ИИ в медицине, такие как конфиденциальность данных, равенство доступа к технологиям и ответственность за принятие реш

### 2️⃣ **Распределённая структура (Distributed)**

👥 Все агенты равны, взаимодействуют напрямую без центрального координатора, согласуя между собой.

In [42]:
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager

# Те же агенты, но без центрального менеджера
researcher = AssistantAgent(name="Researcher", llm_config=llm_config,
    system_message="Собери информацию о применении ИИ в медицине и поделись с другими.")
writer = AssistantAgent(name="Writer", llm_config=llm_config,
    system_message="На основе данных от других агентов напиши черновик.")
editor = AssistantAgent(name="Editor", llm_config=llm_config,
    system_message="Оформи текст в итоговый отчёт и сообщи участникам.")

user = UserProxyAgent(name="User", human_input_mode="NEVER", code_execution_config=False)

group_chat = GroupChat(
    agents=[user, researcher, writer, editor],
    messages=[],
    max_round=12
)

chat_manager = GroupChatManager(groupchat=group_chat, llm_config=llm_config)

user.initiate_chat(chat_manager, message=(
    "Подготовьте совместный отчёт по теме 'Будущее ИИ в медицине'. Обсудите и скоординируйтесь между собой без лидера."
))

User (to chat_manager):

Подготовьте совместный отчёт по теме 'Будущее ИИ в медицине'. Обсудите и скоординируйтесь между собой без лидера.

--------------------------------------------------------------------------------
[autogen.oai.client: 06-23 12:09:41] {696} WARNING - Model mistral-nemo:12b-instruct-2407-q3_K_L is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.

Next speaker: Researcher

[autogen.oai.client: 06-23 12:11:04] {696} WARNING - Model mistral-nemo:12b-instruct-2407-q3_K_L is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
Researcher (to chat_manager):

Введение

Искусственный интеллект (ИИ) все больше интегрируется в различные аспекты нашей жизни, включая здравоохранение. В этом отчете мы рассмотрим текущее состояние применения ИИ в медицине и то, как оно может эволюц

ChatResult(chat_id=None, chat_history=[{'content': "Подготовьте совместный отчёт по теме 'Будущее ИИ в медицине'. Обсудите и скоординируйтесь между собой без лидера.", 'role': 'assistant', 'name': 'User'}, {'content': 'Введение\n\nИскусственный интеллект (ИИ) все больше интегрируется в различные аспекты нашей жизни, включая здравоохранение. В этом отчете мы рассмотрим текущее состояние применения ИИ в медицине и то, как оно может эволюционировать в будущем.\n\nТекущее применение ИИ в медицине\n\n1. Диагностика заболеваний\nОдним из основных применений ИИ в медицине является помощь в диагностике заболеваний. Алгоритмы машинного обучения могут анализировать большие объемы медицинских изображений, таких как рентгеновские снимки и МРТ-сканы, для выявления признаков заболеваний, таких как рак или болезни сердца.\n\n2. Лечение и хирургия\nИИ также используется в лечении и проведении хирургических операций. Роботы-ассистенты могут выполнять сложные хирургические процедуры с большей точностью 

### 3️⃣ **Иерархическая структура (Hierarchical)**

🔝 Один старший агент (руководитель) делегирует задачи промежуточным агентам, а те управляют исполнителями.

In [43]:
# Руководитель
director = AssistantAgent(name="Director", llm_config=llm_config,
    system_message="Ты руководитель проекта. Распределяй задачи между руководителями направлений.")

# Руководители направлений
head_of_research = AssistantAgent(name="ResearchLead", llm_config=llm_config,
    system_message="Ты глава исследований. Получи задачу от директора и подготовь обзор.")
head_of_content = AssistantAgent(name="ContentLead", llm_config=llm_config,
    system_message="Ты глава написания текста. Получи данные и организуй написание и редактуру.")

# Исполнители
researcher = AssistantAgent(name="Researcher", llm_config=llm_config,
    system_message="Собери факты по заданной теме.")
writer = AssistantAgent(name="Writer", llm_config=llm_config, system_message="Напиши черновик.")
editor = AssistantAgent(name="Editor", llm_config=llm_config,
    system_message="Отредактируй текст.")

user = UserProxyAgent(name="User", human_input_mode="NEVER", code_execution_config=False)

group_chat = GroupChat(
    agents=[user, director, head_of_research, head_of_content, researcher, writer, editor],
    messages=[],
    max_round=15
)

chat_manager = GroupChatManager(groupchat=group_chat, llm_config=llm_config)

user.initiate_chat(chat_manager, message=(
    "Подготовьте совместный отчёт по теме 'Будущее ИИ в медицине'. Director управляет ResearchLead и ContentLead, а те координируют исполнителей."
))


User (to chat_manager):

Подготовьте совместный отчёт по теме 'Будущее ИИ в медицине'. Director управляет ResearchLead и ContentLead, а те координируют исполнителей.

--------------------------------------------------------------------------------
[autogen.oai.client: 06-23 12:21:45] {696} WARNING - Model mistral-nemo:12b-instruct-2407-q3_K_L is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.

Next speaker: Director

[autogen.oai.client: 06-23 12:22:08] {696} WARNING - Model mistral-nemo:12b-instruct-2407-q3_K_L is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
Director (to chat_manager):

В качестве руководителя проекта я распределил бы задачи следующим образом:

1. **Director (Руководитель)**
   - Координация всей работы над отчетом.
   - Обеспечение соблюдения сроков и бюджета.
 

ChatResult(chat_id=None, chat_history=[{'content': "Подготовьте совместный отчёт по теме 'Будущее ИИ в медицине'. Director управляет ResearchLead и ContentLead, а те координируют исполнителей.", 'role': 'assistant', 'name': 'User'}, {'content': 'В качестве руководителя проекта я распределил бы задачи следующим образом:\n\n1. **Director (Руководитель)**\n   - Координация всей работы над отчетом.\n   - Обеспечение соблюдения сроков и бюджета.\n   - Контроль качества работы ResearchLead и ContentLead.\n\n2. **ResearchLead (Ведущий исследователь)**\n   - Координирование исполнителей, ответственных за сбор информации о текущем состоянии ИИ в медицине и его будущих возможностях.\n   - Анализ собранных данных и выявление основных трендов и перспектив.\n   - Составление раздела отчета, посвященного техническим аспектам и инновациям в области применения ИИ в медицине.\n\n3. **ContentLead (Ведущий по контенту)**\n   - Координирование исполнителей, ответственных за составление текста отчета.\n   

# Rule based стратегия

In [44]:
rule_based_doctor = AssistantAgent(
    name="RuleBasedDoctor",
    llm_config=llm_config,
    system_message=(
        "Ты врач. Следуй чётким правилам:\n"
        "1. Если боль < 3 месяцев — использовать НПВП.\n"
        "2. Если боль > 3 месяцев — когнитивно-поведенческая терапия и физиотерапия.\n"
        "3. Избегать опиоидов без крайней необходимости."
    )
)

user = UserProxyAgent(name="User", human_input_mode="NEVER", code_execution_config=False)

group_chat = GroupChat(
    agents=[user, rule_based_doctor],
    messages=[],
    max_round=5
)

manager = GroupChatManager(groupchat=group_chat, llm_config=llm_config)

user.initiate_chat(
    manager,
    message="У меня хроническая боль уже полгода. Что вы можете порекомендовать?"
)

User (to chat_manager):

У меня хроническая боль уже полгода. Что вы можете порекомендовать?

--------------------------------------------------------------------------------


GroupChat is underpopulated with 2 agents. Consider setting speaker_selection_method to 'round_robin' or allow_repeat_speaker to False, or use direct communication, unless repeated speaker is desired.


[autogen.oai.client: 06-23 12:45:37] {696} WARNING - Model mistral-nemo:12b-instruct-2407-q3_K_L is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.

Next speaker: RuleBasedDoctor

[autogen.oai.client: 06-23 12:46:00] {696} WARNING - Model mistral-nemo:12b-instruct-2407-q3_K_L is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
RuleBasedDoctor (to chat_manager):

Как врач, я могу порекомендовать вам следующее:

1. Когнитивно-поведенческую терапию (КПТ): КПТ поможет вам справиться с хронической болью, изменив ваше отношение к ней и научив вас управлять болевыми ощущениями.
2. Физиотерапию: Физические упражнения и другие методы физиотерапии помогут вам сохранить подвижность, силу и гибкость, а также уменьшить боль.
3. Медикаментозное лечение: Несмотря на то, что опиоиды не рекомендуются 

GroupChat is underpopulated with 2 agents. Consider setting speaker_selection_method to 'round_robin' or allow_repeat_speaker to False, or use direct communication, unless repeated speaker is desired.


[autogen.oai.client: 06-23 12:46:06] {696} WARNING - Model mistral-nemo:12b-instruct-2407-q3_K_L is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.

Next speaker: RuleBasedDoctor

[autogen.oai.client: 06-23 12:46:10] {696} WARNING - Model mistral-nemo:12b-instruct-2407-q3_K_L is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
RuleBasedDoctor (to chat_manager):



--------------------------------------------------------------------------------


GroupChat is underpopulated with 2 agents. Consider setting speaker_selection_method to 'round_robin' or allow_repeat_speaker to False, or use direct communication, unless repeated speaker is desired.


[autogen.oai.client: 06-23 12:46:15] {696} WARNING - Model mistral-nemo:12b-instruct-2407-q3_K_L is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.

Next speaker: RuleBasedDoctor

[autogen.oai.client: 06-23 12:46:21] {696} WARNING - Model mistral-nemo:12b-instruct-2407-q3_K_L is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
RuleBasedDoctor (to chat_manager):

Пожалуйста, проконсультируйтесь со своим лечащим врачом для получения персонализированных рекомендаций по управлению хронической болью.

--------------------------------------------------------------------------------


GroupChat is underpopulated with 2 agents. Consider setting speaker_selection_method to 'round_robin' or allow_repeat_speaker to False, or use direct communication, unless repeated speaker is desired.


[autogen.oai.client: 06-23 12:46:27] {696} WARNING - Model mistral-nemo:12b-instruct-2407-q3_K_L is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.

Next speaker: RuleBasedDoctor

[autogen.oai.client: 06-23 12:46:31] {696} WARNING - Model mistral-nemo:12b-instruct-2407-q3_K_L is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
RuleBasedDoctor (to chat_manager):



--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (1a086cf9-fef6-4199-8b7d-32d1c03f77fe): Maximum rounds (5) reached


ChatResult(chat_id=None, chat_history=[{'content': 'У меня хроническая боль уже полгода. Что вы можете порекомендовать?', 'role': 'assistant', 'name': 'User'}, {'content': 'Как врач, я могу порекомендовать вам следующее:\n\n1. Когнитивно-поведенческую терапию (КПТ): КПТ поможет вам справиться с хронической болью, изменив ваше отношение к ней и научив вас управлять болевыми ощущениями.\n2. Физиотерапию: Физические упражнения и другие методы физиотерапии помогут вам сохранить подвижность, силу и гибкость, а также уменьшить боль.\n3. Медикаментозное лечение: Несмотря на то, что опиоиды не рекомендуются для длительного использования из-за риска зависимости, в некоторых случаях они могут быть назначены для краткосрочного облегчения сильной боли. Ваш врач может порекомендовать другие виды обезболивающих препаратов, таких как НПВП или противоэпилептические средства.\n4. Здоровый образ жизни: Регулярные физические упражнения, здоровое питание и достаточный сон помогут вам справиться с хроничес

**Рассмотренные концепции – кооперация и конкуренция агентов, архитектурные схемы MAS, а также стратегии поведения – образуют теоретический фундамент для понимания мультиагентных систем.**

В современных исследованиях этим аспектам уделяется первостепенное внимание, поскольку от них зависят коллективные способности агентов решать сложные задачи. 

В частности, ускоренное развитие направления кооперации агентов подчёркивает важность разработки алгоритмов, поощряющих сотрудничество и координацию даже в ситуациях с потенциальным конфликтом интересов. 

Для продвинутых МАС все эти элементы – механизмы кооперации, структура взаимодействия и индивидуальные стратегии – должны проектироваться совместно, дополняя друг друга. Новейшие обзоры (2024–2025 гг.) отмечают, что дальнейший прогресс в области мультиагентных систем связан с решениями проблем масштабируемости (большое число агентов), гетерогенности (разные типы агентов, включая людей), а также с интеграцией методов обучения и рассуждения для более разумного коллективного поведения.

Таким образом, теория, изложенная в данной лекции, не только описывает текущее состояние мультиагентных систем, но и служит основой для новых исследований, направленных на создание эффективных, слаженных и интеллектуальных мультиагентных команд будущего.